See [Scala and jupyter notebook](https://jonnylaw.rocks/posts/2019-04-15-scala-and-jupyter-notebook-with-almond/) for some info about setup.

In [1]:
import $ivy.`com.beachape::enumeratum:1.7.3`, enumeratum.{Enum, EnumEntry}
import $ivy.`com.eed3si9n.expecty::expecty:0.16.0`, com.eed3si9n.expecty.Expecty.expect
import fastparse._
import scala.language.implicitConversions

import $ivy.$                               , enumeratum.{Enum, EnumEntry}

import $ivy.$                                     , com.eed3si9n.expecty.Expecty.expect

import fastparse._
import scala.language.implicitConversions

An expression in µDhall is one of the 9 cases:

1. A constant number of type `Natural`, for example: `123`
2. A constant built-in symbol, for example: `Type`
3. A variable symbol, possibly with an index, for example: `x@n`
4. An exression with an explicit type, for example: `x : t`
5. A lambda function, for example: `λ(a : t) → b`
6. A function type, for example: `∀(a : t) → b`
7. A `let` expression, for example: `let x = a in b`
8. A function application, for example: `f a`
9. A built-in binary operation, for example: `x * y`

In [2]:
// Define the set of built-in symbols supported in µDhall.

sealed abstract class Constant(override val entryName: String) extends EnumEntry {}

object Constant extends Enum[Constant] {
  override def values = findValues

  case object Natural         extends Constant("Natural")
  case object NaturalFold     extends Constant("Natural/fold")
  case object NaturalSubtract extends Constant("Natural/subtract")
  case object Kind            extends Constant("Kind")
  case object Type            extends Constant("Type")
}
import Constant._

defined class Constant
defined object Constant
import Constant._

In [3]:
// Define the set of built-in binary operators supported in µDhall.

sealed abstract class Operator(val name: String) extends EnumEntry

object Operator extends Enum[Operator] {
  val values = findValues
  // These operators work only with values of type Natural.
  case object Plus extends Operator("+")
  case object Times extends Operator("*")
}

defined class Operator
defined object Operator

In [4]:
sealed trait Expr

object Expr {
  // Natural literals, for example 123
  final case class NaturalLiteral(value: Int) extends Expr {
      require(value >= 0)
  }
  // Variables with their de Bruijn indices.
  final case class Variable(name: String, index: Int = 0) extends Expr {
      require (index >= 0)
  }
  // λ(name : tipe) → body  -- Function literal value.
  final case class Lambda(name: String, tipe: Expr, body: Expr) extends Expr

  // ∀(name : tipe) → body  -- Function type.
  final case class Forall(name: String, tipe: Expr, body: Expr) extends Expr

  // let name = subst in body  -- Locally scoped variable definition.
  final case class Let(name: String, subst: Expr, body: Expr) extends Expr

  // body : tipe   -- Expression that is annotated with a type.
  final case class Annotated(body: Expr, tipe: Expr) extends Expr

  // func arg   -- Application of a function to an argument.
  final case class Applied(func: Expr, arg: Expr) extends Expr

  // Built-in constant symbols such as "Natural" or "Type".
  final case class Builtin(constant: Constant) extends Expr

  // Binary operations such as "n + 123".
  final case class BinaryOp(left: Expr, op: Operator, right: Expr) extends Expr
}

cmd4.sc:211: The outer reference in this type test cannot be checked at run time.
  final case class NaturalLiteral(value: Int) extends Expr {
                   ^
cmd4.sc:215: The outer reference in this type test cannot be checked at run time.
  final case class Variable(name: String, index: Int = 0) extends Expr {
                   ^
cmd4.sc:219: The outer reference in this type test cannot be checked at run time.
  final case class Lambda(name: String, tipe: Expr, body: Expr) extends Expr
                   ^
cmd4.sc:222: The outer reference in this type test cannot be checked at run time.
  final case class Forall(name: String, tipe: Expr, body: Expr) extends Expr
                   ^
cmd4.sc:225: The outer reference in this type test cannot be checked at run time.
  final case class Let(name: String, subst: Expr, body: Expr) extends Expr
                   ^
cmd4.sc:228: The outer reference in this type test cannot be checked at run time.
  final case class Annotated(body: Expr,

defined trait Expr
defined object Expr

In [5]:
val test = Expr.BinaryOp(Expr.NaturalLiteral(123), Operator.Plus, Expr.Variable("a", 0))

test: Expr.BinaryOp = BinaryOp(
  left = NaturalLiteral(value = 123),
  op = Plus,
  right = Variable(name = "a", index = 0)
)

In [25]:
implicit class ExprMap(e: Expr) {
    def map(f: Expr => Expr): Expr = e match {
        case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _) => e
/*case Expr.Lambda(name, tipe, body)   => Expr.Lambda(name, tipe.map(f), body.map(f))
        case Expr.Forall(name, tipe, body)   => Expr.Forall(name, tipe.map(f), body.map(f))
        case Expr.Let(name, subst, body)     => Expr.Let(name, subst.map(f), body.map(f))
        case Expr.Annotated(body, tipe)      => Expr.Annotated(body.map(f), tipe.map(f))
        case Expr.Applied(func, arg)         => Expr.Applied(func.map(f), arg.map(f))
        case Expr.BinaryOp(left, op, right)  => Expr.BinaryOp(left.map(f), op, right.map(f))*/
        case Expr.Lambda(name, tipe, body)   => Expr.Lambda(name, f(tipe), f(body))
        case Expr.Forall(name, tipe, body)   => Expr.Forall(name, f(tipe), f(body))
        case Expr.Let(name, subst, body)     => Expr.Let(name, f(subst), f(body))
        case Expr.Annotated(body, tipe)      => Expr.Annotated(f(body), f(tipe))
        case Expr.Applied(func, arg)         => Expr.Applied(f(func), f(arg))
        case Expr.BinaryOp(left, op, right)  => Expr.BinaryOp(f(left), op, f(right))
    }
}

cmd25.sc:3: The outer reference in this type test cannot be checked at run time.
        case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _) => e
                                ^
cmd25.sc:3: The outer reference in this type test cannot be checked at run time.
        case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _) => e
                                                  ^
cmd25.sc:3: The outer reference in this type test cannot be checked at run time.
        case Expr.NaturalLiteral(_) | Expr.Builtin(_) | Expr.Variable(_, _) => e
                                                                     ^
cmd25.sc:10: The outer reference in this type test cannot be checked at run time.
        case Expr.Lambda(name, tipe, body)   => Expr.Lambda(name, f(tipe), f(body))
                        ^
cmd25.sc:11: The outer reference in this type test cannot be checked at run time.
        case Expr.Forall(name, tipe, body)   => Expr.Forall(name, f(tipe), f(body))

defined class ExprMap

In [26]:
object DSL { // Helper methods for creating µDhall values more easily in Scala.
    import Expr._
    import Constant._
    import Operator._
    
    implicit class IntroduceVar(name: String) {
        def ! : Variable = Variable(name)
        def !!(index: Int) : Variable = Variable(name, index)
    }
    implicit class IntroduceNatural(n: Int) {
        def ! : NaturalLiteral = NaturalLiteral(n)
    }
    implicit class IntroduceSymbol(c: Constant) {
        def ! : Expr = Builtin(c)
    }
    implicit class NaturalOps(e: Expr) {
        def +(other: Expr): Expr = BinaryOp(e, Plus, other)
        def *(other: Expr): Expr = BinaryOp(e, Times, other)
    }
    implicit class ExprAnnotate(e: Expr) {
        def :~(tipe: Expr): Expr = Annotated(e, tipe)
        def apply(arg: Expr): Expr = Applied(e, arg)
        // Instead of "let x = e in body" we write body.let(x, e)
        def let(arg: String, subst: Expr): Expr = Let(arg, subst, e)
    }
    implicit class ExprFunc(x: Expr) {
        // Instead of "λ(name : tipe) → body" we write name.! :~ tipe ~> body
        def ~>(body: Expr): Expr = x match {
            case Annotated(Variable(v, 0), tipe) => Lambda(v, tipe, body)
            case _ => throw new Exception(s"Invalid Lambda: argument must be an Annotated name but instead got $x")
        }
        // Instead of "∀(name : tipe) → body" we write name.! :~ tipe :~> body
        def :~>(body: Expr): Expr = x match {
            case Annotated(Variable(v, 0), tipe) => Forall(v, tipe, body)
            case _ => throw new Exception(s"Invalid Forall: argument must be an Annotated name but instead got $x")
        }
    }
}
import DSL._

cmd26.sc:29: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tipe) => Lambda(v, tipe, body)
                          ^
cmd26.sc:29: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tipe) => Lambda(v, tipe, body)
                                   ^
cmd26.sc:34: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tipe) => Forall(v, tipe, body)
                          ^
cmd26.sc:34: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tipe) => Forall(v, tipe, body)
                                   ^


defined object DSL
import DSL._

In [27]:
object Test1 {
    // A simple test.

    val test0 = 1.! + 2.! + 3.!
    val test1 = "n".! + 123.! :~  Natural.!
    val test2 = ("n".! :~ Natural.!) ~> ("n".! + 1.!)
    val test3 = ("n".! :~ Natural.!) :~> Natural.!
    val test4 = test2 :~ test3
    val test5 = test2(test1)
    val test6 = "f".! ("g".! ("x".!))
}

defined object Test1

In [28]:
/*
   The pretty-printer works by computing the "inner" and "outer" binding precedence of each expression.

   - Parentheses are required whenever the outer precedence is below the inner precedence.

   Some examples:

   a   *   b   +   c                 Plus( Times (a, b), c )
      10      20

   (a   +   b)   *   f     (g   c)           Times ( Plus (a, b), Applied(f, Applied(g, c) ) )
       20       10     5,4    5

   f   a     (b   +   c)   +   d       Plus( Applied ( Applied (f, a), Plus (b, c) ), d )
     5   5,4     20       20

   λ(a : Natural) → λ(b : Natural)  →  f   a   +   b    Lambda ( a, Natural, Lambda(b, Natural, Plus (Applied(f, a), b) ) )
       8          3     8           3    5    20

   (λ(a : Natural)  →  a)   b         Applied (Lambda (a, Natural, a), b)
        8           3     5  

   - Each of the Expr constructors has an overall outer precedence and a separate inner precedence for each Expr argument.

   - Precedence values must be specified separately for each constructor and each argument.

   Applied(f, a) has outer precedence 5 and inner precedence 

   - Binary operations have equal outer and inner precedence values. This is the "precedence of the operation".

   - Other constructors sometimes have unequal outer and inner precedence values.
*/

def precedence(op: Operator): Int = op match {
    case Operator.Plus => 20
    case Operator.Times => 10
}

// Return (outer, List(inner1, inner2, ...)) for each constructor that may have Expr arguments.
def precedence(e: Expr): (Int, List[Int]) = e match {
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
    case Expr.Lambda(name, tipe, body) => (50, List(50, 50))
    case Expr.Forall(name, tipe, body) => (50, List(50, 50))
    case Expr.Let(name, subst, body) => (50, List(50, 50))
    case Expr.Annotated(body, tipe) => (8, List(7, 60))   //   ( 1 : Natural ) : Natural : Type
    case Expr.Applied(func, arg) => (5, List(4, 4))      //   f (g x)
    case Expr.BinaryOp(_, op, _) =>
        val prec = precedence(op)
        (prec, List(prec, prec)) // For binary operators, all 3 precedence priorities are equal.
}

def inPrecedence(expr: String, innerPrec: Int, outerPrec: Int): String =
    if (innerPrec > outerPrec) s"($expr)" else expr

def prettyprint(e: Expr, outside: Int = 100): String = {
  val (outer, inner) = precedence(e)

def printpair(prefix: String, left: Expr, middle: String, right: Expr): String =
    prefix + prettyprint(left, inner(0)) + middle + prettyprint(right, inner(1))

  val exprPrinted = e match {
    case Expr.NaturalLiteral(value) => value.toString
    case Expr.Variable(name, index) => name + (if (index != 0) s"@$index" else "")
    case Expr.Builtin(constant) => constant.entryName
    case Expr.Lambda(name, tipe, body) => printpair(s"λ($name : ", tipe, ") → ", body)
    case Expr.Forall(name, tipe, body) => printpair(s"∀($name : ", tipe, ") → ", body)
    case Expr.Let(name, subst, body) => printpair(s"let $name = ", subst, " in ", body)
    case Expr.Annotated(body, tipe) => printpair("", body, " : ", tipe)
    case Expr.Applied(func, arg) => printpair("", func, " ", arg)
    case Expr.BinaryOp(left, op, right) => printpair("", left, " " + op.name + " ", right)
  }

  inPrecedence(exprPrinted, outer, outside)
}

cmd28.sc:41: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
                            ^
cmd28.sc:41: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
                                               ^
cmd28.sc:41: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
                                                                    ^
cmd28.sc:42: The outer reference in this type test cannot be checked at run time.
    case Expr.Lambda(name, tipe, body) => (50, List(50, 50))
                    ^
cmd28.sc:43: The outer reference in this type test cannot be checked at run time.
    case Expr

defined function precedence
defined function precedence
defined function inPrecedence
defined function prettyprint

In [29]:
// Test the pretty-printer.
Seq(
        ( Test1.test0 -> "1 + 2 + 3"),
        ( Test1.test1 -> "(n + 123) : Natural"),
        ( Test1.test2 -> "λ(n : Natural) → n + 1"),
        ( Test1.test3 -> "∀(n : Natural) → Natural"),
        ( Test1.test4 -> "(λ(n : Natural) → n + 1) : ∀(n : Natural) → Natural"),
        ( Test1.test5 -> "(λ(n : Natural) → n + 1) ((n + 123) : Natural)"),
        ( Test1.test6 -> "f (g x)"),
).zipWithIndex.foreach { case ((expr, expected), i) => expect(i >= 0 && prettyprint(expr) == expected) }

"Tests passed for prettyprint()."

res29_1: String = "Tests passed for prettyprint()."